In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [3]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [4]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [5]:
line = Line([y, z])

In [6]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [7]:
SE6 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f: -c*(5*c+e)/(c+e)})

In [8]:
SE6

4*x^2*z*c^2 - 8*x*y*z*c^2 + 2*x*z^2*c^2 - 6*x*y*t*c^2 + 2*y^2*t*c^2 + 4*y*z*t*c^2 + 2*y*t^2*c^2 - 10*x^2*y*c*e + 10*x*y^2*c*e + 4*x^2*z*c*e - 8*x*y*z*c*e + 2*x*z^2*c*e + 4*x*y*t*c*e - 8*y^2*t*c*e + 4*y*z*t*c*e + 2*y*t^2*c*e - 2*x^2*y*e^2 + 2*x*y^2*e^2 + 2*x*y*t*e^2 - 2*y^2*t*e^2

In [8]:
%time adm_SE6 = SE6.find_admissible_projectivities()

CPU times: user 410 ms, sys: 137 ms, total: 547 ms
Wall time: 1.1 s


In [9]:
len(adm_SE6)

48

In [10]:
%time simm_SE6 = SE6.find_simmetries(adm_SE6)

CPU times: user 305 ms, sys: 73 ms, total: 378 ms
Wall time: 442 ms


In [11]:
len(simm_SE6)

24

In [12]:
SE6.find_conditions_for_subfamilies(adm_SE6, simm_SE6)

[]

In [13]:
perms = []
for el in simm_SE6:
    perms.append(Permutation(SE6.apply_proj_to_eck(el)).to_cycles())

In [14]:
gp = PermutationGroup(perms)

In [15]:
gp.order()

24

In [16]:
gp.structure_description()

'S4'

In [17]:
nperms = [[cycle for cycle in perm if len(cycle)>1] for perm in perms]

In [18]:
for gen in gp.minimal_generating_set():
    print(nperms.index(gen.cycle_tuples()))

13
14


In [20]:
gen1 = simm_SE6[13]
gen2 = simm_SE6[14]

In [21]:
eck_labels = SE6.eckardt_points_labels

In [22]:
eck = SE6.eckardt_points

In [23]:
for i in range(4):
    for j in range(i+1, 5):
        for k in range(j+1, 6):
            point1 = eck[i]
            point2 = eck[j]
            point3 = eck[k]
            if matrix([point1, point2, point3]).minors(3) == [0,0,0,0]:
                print(eck_labels[i], eck_labels[j], eck_labels[k])

['E1', 'G4', 'F14'] ['E2', 'G1', 'F12'] ['E4', 'G2', 'F24']
['E1', 'G4', 'F14'] ['E2', 'G3', 'F23'] ['F13', 'F24', 'F56']
['E2', 'G1', 'F12'] ['E3', 'G4', 'F34'] ['F13', 'F24', 'F56']
['E2', 'G3', 'F23'] ['E3', 'G4', 'F34'] ['E4', 'G2', 'F24']


In [24]:
new_lines = []
lines_plucker = [line.plucker for line in SE6.lines]
for i in range(len(eck)):
    point1 = eck[i]
    for j in range(i+1, len(eck)):
        point2 = eck[j]
        new_line_plucker = Point(matrix([point1, point2]).minors(2))
        if new_line_plucker not in lines_plucker and new_line_plucker not in [line.plucker for line in new_lines]:
            planes = get_two_planes_containing_line(new_line_plucker)
            new_lines.append(Line(planes, [point1, point2], new_line_plucker))

In [25]:
for line in new_lines:
    print(line.apply_proj(gen1) in new_lines, line.apply_proj(gen2) in new_lines)

True True
True True
True True
True True


In [26]:
keys = list(SE6.cl_lines.keys())

In [27]:
perms_lines = []
for simm in simm_SE6:
    perms_lines.append(Permutation(SE6.apply_proj_to_lines(simm)).to_cycles())

In [28]:
Gr = PermutationGroup(perms_lines)

In [29]:
Gr.order()

24

In [30]:
Gr.structure_description()

'S4'

In [31]:
Gr.minimal_generating_set()

[(1,8,13)(2,10,19)(3,27,9)(4,7,15)(5,24,11)(6,23,12)(14,18,22)(16,20,25)(17,21,26),
 (1,3,15,22)(2,19)(4,13,14,18)(5,6)(7,27,9,8)(11,23,21,16)(12,24,20,17)(25,26)]

In [32]:
[el.order() for el in Gr.normal_subgroups()]

[24, 12, 4, 1]